In [6]:
from llama_index.core import SimpleDirectoryReader
from openai import OpenAI
from dotenv import load_dotenv

import json
import os

In [88]:
load_dotenv()

EXTRACT_ROOLS_MODEL = 'meta-llama/Llama-3.3-70B-Instruct'
DEEPINFRA_TOKEN = os.environ.get('DEEPINFRA_API_TOKEN')

In [96]:
documents = SimpleDirectoryReader(input_dir="./data", required_exts=[".docx"]).load_data()
doc_texts = [document.text for document in documents]

In [97]:
print(doc_texts[0])

Документ предоставлен КонсультантПлюс




Зарегистрировано в Минюсте России 5 августа 2022 г. N 69536





МИНИСТЕРСТВО ТРУДА И СОЦИАЛЬНОЙ ЗАЩИТЫ РОССИЙСКОЙ ФЕДЕРАЦИИ



ФЕДЕРАЛЬНАЯ СЛУЖБА ПО ТРУДУ И ЗАНЯТОСТИ



ПРИКАЗ

от 8 июля 2022 г. N 173



ОБ УТВЕРЖДЕНИИ ТРЕБОВАНИЙ

К ИНФОРМАЦИИ, РАЗМЕЩАЕМОЙ НА ЕДИНОЙ ЦИФРОВОЙ ПЛАТФОРМЕ

В СФЕРЕ ЗАНЯТОСТИ И ТРУДОВЫХ ОТНОШЕНИЙ "РАБОТА В РОССИИ",

А ТАКЖЕ ПОРЯДКА ПРОВЕДЕНИЯ ОЦЕНКИ ИНФОРМАЦИИ, РАЗМЕЩАЕМОЙ

НА ЕДИНОЙ ЦИФРОВОЙ ПЛАТФОРМЕ В СФЕРЕ ЗАНЯТОСТИ И ТРУДОВЫХ

ОТНОШЕНИЙ "РАБОТА В РОССИИ", НА ПРЕДМЕТ СООТВЕТСТВИЯ

ТРЕБОВАНИЯМ К ИНФОРМАЦИИ, РАЗМЕЩАЕМОЙ НА ЕДИНОЙ ЦИФРОВОЙ

ПЛАТФОРМЕ В СФЕРЕ ЗАНЯТОСТИ И ТРУДОВЫХ ОТНОШЕНИЙ

"РАБОТА В РОССИИ"



В соответствии с абзацами третьим и четвертым пункта 3 постановления Правительства Российской Федерации от 13 мая 2022 г. N 867 "О единой цифровой платформе в сфере занятости и трудовых отношений "Работа в России" (Официальный интернет-портал правовой информации (www.pravo.gov.ru), 2022, 16 мая, N 0001202205

In [99]:
prompt = """
    Ты — эксперт по анализу документов. Извлеки из текста ниже все правила оформления резюме и преобразуй их в структурированный JSON-список. 

    **Инструкции:**
    1. Каждое правило должно иметь уникальный `id`.
    2. В поле `condition` кратко опиши условие, которое нужно проверить.

    **Пример вывода:**
    {{
      "rules": [
        {{
          "id": "rule_1",
          "condition": "Резюме содержит раздел 'Опыт работы'",
        }}
      ]
    }}

    **Текст документа:**
    {doc_text}
    """

In [101]:
def get_structured_rules(document_text):
    client = OpenAI(
    api_key=DEEPINFRA_TOKEN,
    base_url="https://api.deepinfra.com/v1/openai",
    )

    
    response = client.chat.completions.create(
        model=EXTRACT_ROOLS_MODEL,
        messages=[{"role": "user", "content": prompt.format(doc_text=document_text)}],
        response_format={"type": "json_object"},
        temperature=0
    )
    return json.loads(response.choices[0].message.content)

In [102]:
rools_dicts = [get_structured_rules(document_text=doc_text) for doc_text in doc_texts]

In [106]:
all_rools = rools_dicts[0]['rules'] + rools_dicts[1]['rules']

In [108]:
for i in range(len(all_rools)):
    all_rools[i]['id'] = f'rule_{i+1}'

In [109]:
all_rools

[{'id': 'rule_1',
  'condition': 'Информация о работодателе не должна содержать сведений, не относящихся к работодателю'},
 {'id': 'rule_2',
  'condition': 'Информация о работодателе должна соответствовать информации, содержащейся в Едином государственном реестре юридических лиц или Едином государственном реестре индивидуальных предпринимателей'},
 {'id': 'rule_3',
  'condition': 'Информация о вакансии не должна содержать сведений о прямом или косвенном ограничении прав или об установлении прямых или косвенных преимуществ'},
 {'id': 'rule_4',
  'condition': 'Информация о резюме не должна содержать сведений, не относящихся к соискателю'},
 {'id': 'rule_5',
  'condition': 'Информация не должна содержать слова и выражения, не соответствующие нормам современного русского литературного языка'},
 {'id': 'rule_6',
  'condition': 'Информация должна соответствовать требованиям, утвержденным приказом Федеральной службы по труду и занятости'},
 {'id': 'rule_7',
  'condition': 'Информация должна б

In [110]:
to_drop = {
    'rule_2',
    'rule_6',
    'rule_7',
    'rule_8',
    'rule_9',
    'rule_10',
    'rule_12',
    'rule_13',
    'rule_14',
    'rule_15',
    'rule_16',
    'rule_17',
    'rule_19',
    'rule_20',
    'rule_21'
}

In [111]:
filtered_rools = [rool for rool in all_rools if rool['id'] not in to_drop]

In [113]:
for i in range(len(filtered_rools)):
    filtered_rools[i]['id'] = f'rule_{i+1}'

In [114]:
filtered_rools

[{'id': 'rule_1',
  'condition': 'Информация о работодателе не должна содержать сведений, не относящихся к работодателю'},
 {'id': 'rule_2',
  'condition': 'Информация о вакансии не должна содержать сведений о прямом или косвенном ограничении прав или об установлении прямых или косвенных преимуществ'},
 {'id': 'rule_3',
  'condition': 'Информация о резюме не должна содержать сведений, не относящихся к соискателю'},
 {'id': 'rule_4',
  'condition': 'Информация не должна содержать слова и выражения, не соответствующие нормам современного русского литературного языка'},
 {'id': 'rule_5',
  'condition': 'Резюме содержит информацию о трудовой деятельности'},
 {'id': 'rule_6', 'condition': 'Резюме содержит информацию об образовании'},
 {'id': 'rule_7',
  'condition': 'Название резюме содержит профессию/должность, на которую претендует соискатель'},
 {'id': 'rule_8', 'condition': 'Название резюме не содержит сокращений'},
 {'id': 'rule_9', 'condition': 'Резюме не содержит опечаток/ошибок'},
 

In [1]:
picked_changed_rools = [
    {'id': 'rule_1', 'condition': 'Резюме должно содержать информацию о трудовой деятельности если желаемая должность требует опыта'},
    {'id': 'rule_2', 'condition': 'Резюме должно содержать профессию/должность или должности, на которые претендует соискатель, должность должжна быть реальной профессией'},
    # {'id': 'rule_3', 'condition': 'Резюме не должно содержать опечаток/ошибок, однако допускаются пунктуационные ошибки и повторения в тексте'},
    {'id': 'rule_4', 'condition': 'Резюме должно быть заполнено на русском языке'},
    {'id': 'rule_5', 'condition': 'Информация о работодателе не должна содержать сведений, не относящихся к работодателю, если работодатель ИП это не нарушение правил'},
    {'id': 'rule_6', 'condition': 'Информация о резюме не должна содержать сведений, не относящихся к соискателю'},
    {'id': 'rule_7', 'condition': 'Информация не должна содержать слова и выражения, не соответствующие нормам современного русского литературного языка, допустимы стилистические неточности, но главное чтобы не было матов и токсичности'},
]

In [2]:
len(str(picked_changed_rools))

894

In [7]:
with open('data/resume_rools.json', 'w') as f:
    json.dump(picked_changed_rools, f)